***
## DA 320 

### Assignment: Detecting Outliers MongoDB
#### Student: Natalia Sadkov 
#### Source: code provided by Professor Spence - https://github.com/tspence/data-analysis-class-resources/blob/main/jupyter/Basics%20of%20Loading%20Data.ipynb
##### Date: October, 26 2022

***


In [1]:
#!/usr/bin/python
#import libraries
import urllib3
import certifi

In [6]:
#read data from CVS code
import pandas as pd

# Read this file into memory
imdb = pd.read_csv("/Users/natasha/Desktop/Fall 2022/DA320/imdb_superhero_2.csv")

# Show a sample of a few columns of data
imdb.head()

,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,77631,Grease,110.0,7.2,265183,TV-14::(D),6/13/1978,6000000.0,60759.0,394955690.0,"Comedy, Musical, Romance","John Travolta, Olivia Newton-John, Stockard Ch...",Randal Kleiser,"Allan Carr, Neil A. Machlis, Robert Stigwood","Paramount Pictures, Robert Stigwood Organizati..."
1,78346,Superman,143.0,7.4,172769,TV-PG::(LV),12/10/1978,55000000.0,7465343.0,166200000.0,"Action, Adventure, Sci-Fi","Marlon Brando, Gene Hackman, Christopher Reeve...",Richard Donner,"Charles Greenlaw, Richard Lester, Alexander Sa...","Dovemead Films, Film Export A.G., Internationa..."
2,77416,The Deer Hunter,183.0,8.1,334827,R,12/8/1978,15000000.0,NaN,NaN,"Drama, War","Robert De Niro, John Cazale, John Savage, Chri...",Michael Cimino,"Joann Carelli, Michael Cimino, Michael Deeley,...","EMI Films, Universal Pictures"
3,77651,Halloween,91.0,7.7,267109,TV-14,10/25/1978,300000.0,NaN,70000000.0,"Horror, Thriller","Donald Pleasence, Jamie Lee Curtis, Nancy Kyes...",John Carpenter,"Moustapha Akkad, John Carpenter, Debra Hill, K...","Compass International Pictures, Falcon Interna..."
4,77975,National Lampoon's Animal House,109.0,7.4,119917,"TV-14::(DLSV, TV Rating.)",7/27/1978,3000000.0,201747.0,3371006.0,Comedy,"Tom Hulce, Stephen Furst, Mark Metcalf, Mary L...",John Landis,"Ivan Reitman, Matty Simmons","Universal Pictures, Oregon Film Factory, Stage..."


***
### Retrieve connection strings, passwords, or secrets from a JSON file 
***

In [3]:
import json

# Demonstration of how to load a file that contains secrets without accidentally leaking those secrets
with open('/Users/natasha/Desktop/Fall 2022/DA320/Password MongoDB/credentials.json') as f:
    data = json.load(f)

    # If you want your data to be secure, don't print this variable out!
    # Jupyter will retain a cached version of any printed data and it can be
    # accidentally committed to version control.
secret_key = data['mongodb']

# We can safely print the length of the secret key. That won't leak any sensitive information.
print(f"My secret key is {len(secret_key)} characters in length.")

My secret key is 68 characters in length.


***
### Connect to a MongoDB Server to DA-320 database
***

In [4]:
import pymongo
import certifi

# Once you have retrieved your connection string from a secrets file, use it here
mongo_connection_string = data['mongodb']

# Connect to the database using known good certificates
client = pymongo.MongoClient(mongo_connection_string, tlsCAFile=certifi.where())

# Fetch the database named "DA-320"
da320_database = client['DA-320']

# Within the database we have "collections". Think of them as tables in SQL.
allCollections = da320_database.list_collection_names()

# Here is the list of collections within my database
print(f"Using MongoDB version {client.server_info()['version']}.")
print(f"This database has the collections {allCollections}")

Using MongoDB version 5.0.13.
This database has the collections ['Metacritics', 'imdb']


***
### Fetch imdb collection from a MongoDB Server
***

In [7]:
# Retrieve all records from a collection - this can be a large amount of data!
cursor = da320_database["imdb"].find()

# Convert this information into a Pandas dataframe
imdb = pd.DataFrame(cursor)

# Make sure we've read the information correctly
imdb.head()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,634b797036c53874680c5c86,78492,The Wild Geese,134,6.7,12977,R,6/28/1978,10000000,NULL,NULL,"Action, Adventure, Drama, Thriller, War","Richard Burton, Roger Moore, Richard Harris, H...",Andrew V. McLaglen,"Chris Chrisafis, Erwin C. Dietrich, Euan Lloyd...","Euan Lloyd Productions, Richmond Film Producti..."
1,634b797036c53874680c5c6a,78111,Pretty Baby,110,6.5,10881,TV-MA,1/1/1978,NULL,NULL,NULL,Drama,"Brooke Shields, Keith Carradine, Susan Sarando...",Louis Malle,"Louis Malle, Polly Platt",Paramount Pictures
2,634b797036c53874680c5c62,78346,Superman,143,7.4,172769,TV-PG::(LV),12/10/1978,55000000,7465343,166200000,"Action, Adventure, Sci-Fi","Marlon Brando, Gene Hackman, Christopher Reeve...",Richard Donner,"Charles Greenlaw, Richard Lester, Alexander Sa...","Dovemead Films, Film Export A.G., Internationa..."
3,634b797036c53874680c5c67,77413,Death on the Nile,140,7.2,34243,PG,6/20/1978,7920000,NULL,784489,"Crime, Drama, Mystery","Peter Ustinov, Jane Birkin, Lois Chiles, Bette...",John Guillermin,"John Brabourne, Richard Goodwin, Norton Knatch...","Mersham Productions Ltd., EMI Films"
4,634b797036c53874680c5c63,77416,The Deer Hunter,183,8.1,334827,R,12/8/1978,15000000,NULL,NULL,"Drama, War","Robert De Niro, John Cazale, John Savage, Chri...",Michael Cimino,"Joann Carelli, Michael Cimino, Michael Deeley,...","EMI Films, Universal Pictures"


***
### Checking query subset of data from MongoDB Server DA-320 database
***

In [8]:
import pandas as pd

# Define your query in MongoDB Compass, on the website, or using any other available MongoDB program
query = { "title": { "$regex": "(Wild|baby)" } }

# Execute this query and produce a cursor
cursor = da320_database["imdb"].find(query)

# Convert this information into a Pandas dataframe
imdb = pd.DataFrame(cursor)

# Make sure we've read the information correctly
imdb.head()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,634b797036c53874680c5c86,78492,The Wild Geese,134,6.7,12977,R,6/28/1978,10000000,NULL,NULL,"Action, Adventure, Drama, Thriller, War","Richard Burton, Roger Moore, Richard Harris, H...",Andrew V. McLaglen,"Chris Chrisafis, Erwin C. Dietrich, Euan Lloyd...","Euan Lloyd Productions, Richmond Film Producti..."
1,634b797136c53874680c60bc,84904,Wild Style,82,7,2630,R,6/20/1982,NULL,NULL,NULL,"Drama, Music","Lee Quiñones, Lady Pink, Fab 5 Freddy, Patti A...",Charlie Ahearn,"Charlie Ahearn, Jane Dickson, Fab 5 Freddy",Wild Style
2,634b797136c53874680c62dc,87068,Code Name: Wild Geese,101,5.1,1018,R,10/5/1984,NULL,NULL,NULL,Action,"Lewis Collins, Lee Van Cleef, Ernest Borgnine,...",Antonio Margheriti,"Gianfranco Couyoumdjian, Erwin C. Dietrich","Ascot Film, Gico Cinematografica S.r.l."
3,634b797136c53874680c6257,88402,The Wild Life,96,6,2778,R,9/28/1984,NULL,4001000,NULL,"Comedy, Drama","Chris Penn, Ilan Mitchell-Smith, Eric Stoltz, ...",Art Linson,"Cameron Crowe, C.O. Erickson, Art Linson, Don ...",Universal Pictures
4,634b797136c53874680c6380,90323,Wild Geese II,125,4.8,1393,R,5/31/1985,NULL,NULL,NULL,"Action, Adventure, Thriller","Scott Glenn, Barbara Carrera, Edward Fox, Laur...",Peter R. Hunt,"Chris Chrisafis, Euan Lloyd, Anton Eric Scotoni","Frontier Films, Thorn EMI Screen Entertainment"


***
### Convert Data - release_date to date, budget to numeric, runtime to numeric
***

In [29]:
# Transform and standardize the columns
imdb.release_date=pd.to_datetime(imdb.release_date,errors = 'coerce')
imdb.budget = pd.to_numeric(imdb.budget, errors='coerce')
imdb.runtime=pd.to_numeric(imdb.runtime,errors='coerce')
# cheking result
imdb.head()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,634b797036c53874680c5c86,78492,The Wild Geese,134,6.7,12977,R,1978-06-28,134,NULL,NULL,"Action, Adventure, Drama, Thriller, War","Richard Burton, Roger Moore, Richard Harris, H...",Andrew V. McLaglen,"Chris Chrisafis, Erwin C. Dietrich, Euan Lloyd...","Euan Lloyd Productions, Richmond Film Producti..."
1,634b797136c53874680c60bc,84904,Wild Style,82,7,2630,R,1982-06-20,82,NULL,NULL,"Drama, Music","Lee Quiñones, Lady Pink, Fab 5 Freddy, Patti A...",Charlie Ahearn,"Charlie Ahearn, Jane Dickson, Fab 5 Freddy",Wild Style
2,634b797136c53874680c62dc,87068,Code Name: Wild Geese,101,5.1,1018,R,1984-10-05,101,NULL,NULL,Action,"Lewis Collins, Lee Van Cleef, Ernest Borgnine,...",Antonio Margheriti,"Gianfranco Couyoumdjian, Erwin C. Dietrich","Ascot Film, Gico Cinematografica S.r.l."
3,634b797136c53874680c6257,88402,The Wild Life,96,6,2778,R,1984-09-28,96,4001000,NULL,"Comedy, Drama","Chris Penn, Ilan Mitchell-Smith, Eric Stoltz, ...",Art Linson,"Cameron Crowe, C.O. Erickson, Art Linson, Don ...",Universal Pictures
4,634b797136c53874680c6380,90323,Wild Geese II,125,4.8,1393,R,1985-05-31,125,NULL,NULL,"Action, Adventure, Thriller","Scott Glenn, Barbara Carrera, Edward Fox, Laur...",Peter R. Hunt,"Chris Chrisafis, Euan Lloyd, Anton Eric Scotoni","Frontier Films, Thorn EMI Screen Entertainment"


***
### Plotting friendly currency numbers
***

In [32]:
from matplotlib import pyplot

dataset = { “data”: “US” }
def format_currency(amount, _):
    if amount > 1_000_000_000_000:
        return f"${round(amount / 1_000_000_000_000)}T"
    elif amount > 1_000_000_000:
        return f"${round(amount / 1_000_000_000)}B"
    elif amount > 1_000_000:
        return f"${round(amount / 1_000_000)}M"
    elif amount > 1_000:
        return f"${round(amount / 1_000)}K"
    else:
        return f"${round(amount)}"

pyplot.title('Price of a few major purchases')
pyplot.xlabel('Product')
pyplot.ylabel('Price')
pyplot.scatter(dataset['product'], dataset['cost'])
pyplot.gca().yaxis.set_major_formatter(format_currency)
pyplot.show()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,634b797036c53874680c5c86,78492,The Wild Geese,134,6.7,12977,R,1978-06-28,134,NULL,NULL,"Action, Adventure, Drama, Thriller, War","Richard Burton, Roger Moore, Richard Harris, H...",Andrew V. McLaglen,"Chris Chrisafis, Erwin C. Dietrich, Euan Lloyd...","Euan Lloyd Productions, Richmond Film Producti..."
1,634b797136c53874680c60bc,84904,Wild Style,82,7,2630,R,1982-06-20,82,NULL,NULL,"Drama, Music","Lee Quiñones, Lady Pink, Fab 5 Freddy, Patti A...",Charlie Ahearn,"Charlie Ahearn, Jane Dickson, Fab 5 Freddy",Wild Style
2,634b797136c53874680c62dc,87068,Code Name: Wild Geese,101,5.1,1018,R,1984-10-05,101,NULL,NULL,Action,"Lewis Collins, Lee Van Cleef, Ernest Borgnine,...",Antonio Margheriti,"Gianfranco Couyoumdjian, Erwin C. Dietrich","Ascot Film, Gico Cinematografica S.r.l."
3,634b797136c53874680c6257,88402,The Wild Life,96,6,2778,R,1984-09-28,96,4001000,NULL,"Comedy, Drama","Chris Penn, Ilan Mitchell-Smith, Eric Stoltz, ...",Art Linson,"Cameron Crowe, C.O. Erickson, Art Linson, Don ...",Universal Pictures
4,634b797136c53874680c6380,90323,Wild Geese II,125,4.8,1393,R,1985-05-31,125,NULL,NULL,"Action, Adventure, Thriller","Scott Glenn, Barbara Carrera, Edward Fox, Laur...",Peter R. Hunt,"Chris Chrisafis, Euan Lloyd, Anton Eric Scotoni","Frontier Films, Thorn EMI Screen Entertainment"
